# Simple example 2

In [3]:
import sys
import os
# to run the example, set the following path to the folder path of qthought on your machine
sys.path.append(os.path.abspath('/Users/nuri/qthought/qthought')) 

import qthought.utils as ut
from qthought.protocol import Protocol, ProtocolStep
from qthought.quantumsystem import QuantumSystem
from qthought.agents import Agent
from qthought.interpretations.copenhagen_theory import *
from projectq.ops import H, CNOT

In [27]:
def action1(qsys:QuantumSystem):
    H | qsys['alice_qubit']
    CNOT | (qsys["alice_qubit"], qsys["bob_qubit"])
    
step1 = ProtocolStep(domain={'Qubit': ['alice_qubit','bob_qubit']},
                     descr = 'Perpare a bell state of Alices and Bobs qubit ',
                     time  = 0,
                     action = action1)

def action2(qsys:QuantumSystem):
    observe(qsys['Alice_memory'],qsys['alice_qubit'])
    
step2 = ProtocolStep(domain={'Qubit': 'alice_qubit',
                             'Agent(1,1)': 'Alice'},
                     descr = 'Alice observes s',
                     time  = 1,
                     action = action2)

def action3(qsys:QuantumSystem):
    qsys['Alice'].make_inference()
    
step3 = ProtocolStep(domain={'Qubit': 'alice_qubit',
                             'Agent(1,1)': 'Alice'},
                     descr = 'Alice makes an inference about Bobs measurement',
                     time  = 2,
                     action = action3)

def action4(qsys:QuantumSystem):
    observe(qsys['Bob_memory'],qsys['bob_qubit'])
    
step4 = ProtocolStep(domain={'Qubit': 'bob_qubit',
                             'AgentMemory(1)': 'Bob'},
                     descr = 'Bob observes s',
                     time  = 3,
                     action = action4)

In [28]:
p = Protocol()
p += step1
p += step2
p += step3
p += step4

In [29]:
p

Step 0: Perpare a bell state of Alices and Bobs qubit (t:0)
Step 1: Alice observes s(t:1)
Step 2: Alice makes an inference about Bobs measurement(t:2)
Step 3: Bob observes s(t:3)

Requirements: 
------------------------------
Qubit             ['alice_qubit', 'bob_qubit']
Agent(1,1)        ['Alice']
AgentMemory(1)    ['Bob']

In [36]:
Alice_inference_table = TA = forward_inference(protocol=p, 
                                               subsys_x='Alice_memory', t_x=1, 
                                               subsys_y='Bob_memory', t_y=3, 
                                               silent=True)

(Note: This is the (slow) Python simulator.)


/Users/nuri/qthought/qthought/qthought/agents.py:214: UserWarning: make_inference called without setting an inference_table
  warn('make_inference called without setting an inference_table')
/Users/nuri/qthought/qthought/qthought/agents.py:214: UserWarning: make_inference called without setting an inference_table
  warn('make_inference called without setting an inference_table')


In [42]:
Alice_inference_table

In:(Alice_memory:t1)  |  Out: (Bob_memory:t3)
----------------------------------------------------
           0          |        [0]
           1          |        [1]

In [51]:
qsys = QuantumSystem(p.get_requirements())
qsys["Alice"].set_inference_table(Alice_inference_table, no_prediction_state=1)
qsys['Alice'].prep_inference()
p.run(qsys)

(Note: This is the (slow) Python simulator.)
Require Qubit alice_qubit
Require Qubit bob_qubit
Require Agent(1,1) Alice
Require AgentMemory(1) Bob
0 Perpare a bell state of Alices and Bobs qubit  t:0
State:
0.71|0100000> + 0.71|0100011>
1 Alice observes s t:1
State:
0.71|0100000> + 0.71|0100111>
2 Alice makes an inference about Bobs measurement t:2
State:
0.71|0100000> + 0.71|0101111>
3 Bob observes s t:3
State:
0.71|0100000> + 0.71|1101111>


Exception ignored in: <function Qubit.__del__ at 0x114243488>
Traceback (most recent call last):
  File "/Users/nuri/.local/lib/python3.7/site-packages/projectq/types/_qubit.py", line 135, in __del__
    self.engine.deallocate_qubit(weak_copy)
  File "/Users/nuri/.local/lib/python3.7/site-packages/projectq/cengines/_basics.py", line 153, in deallocate_qubit
    tags=[DirtyQubitTag()] if is_dirty else [])])
  File "/Users/nuri/.local/lib/python3.7/site-packages/projectq/cengines/_main.py", line 288, in send
    raise compact_exception  # use verbose=True for more info
RuntimeError: Qubit has not been measured / uncomputed. Cannot access its classical value and/or deallocate a qubit in superposition!
 raised in:
'  File "/Users/nuri/.local/lib/python3.7/site-packages/projectq/backends/_sim/_pysim.py", line 139, in get_classical_value'
'    raise RuntimeError("Qubit has not been measured / "'
Exception ignored in: <function Qubit.__del__ at 0x114243488>
Traceback (most recent call last):
